In [41]:
import numpy as np
import datetime as dt
from astropy import units as u
from astropy.coordinates import Angle
from astropy.coordinates import Longitude
import pint as pt
from pint import UnitRegistry
import scipy.optimize
import emcee

In [23]:
a = Longitude('-20d')+Longitude('-355d')
print(a)
ureg = UnitRegistry()
ureg.Quantity(10.0,ureg.degC)
ureg.Quantity(10.0,ureg.mbar)

345d00m00s


<Quantity(10.0, 'millibar')>

In [72]:
def compute_Zo(latA, lonA, GHA, DEC):
    # compute LHA from GHA and lonA
    LHA = GHA + lonA
    Zo_rad = np.arcsin(np.sin(DEC.radian)*np.sin(latA.radian) + np.cos(latA.radian)*np.cos(DEC.radian)*np.cos(LHA.radian))
    return Angle(Zo_rad, unit=u.radian)


def dip_correction(h):
    dip_corr = Angle(-0.0293*np.sqrt(h.to_base_units().magnitude), unit=u.deg)
    return dip_corr

def index_correction():
    return Angle('-0d2.3m')

def atmo_correction(Ha = Angle('15d'), P = ureg.Quantity(1010.0,ureg.mbar), T = ureg.Quantity(10.0,ureg.degC)):

    Pmb = P.to(ureg.mbar)
    TdegC = T.to(ureg.degC)

    f = 0.28*Pmb.magnitude/(TdegC.magnitude + 273)
    Ro = -0.0167/np.tan(np.pi/180.0*(Ha.deg + 7.31/(Ha.deg + 4.4)))
    return Angle(Ro*f,unit=u.deg)

def est_longitude(latA, lonA, GHA, DEC, Hs, h = ureg.Quantity(10.0,ureg.feet)):
    # use a root-finding algorithm to compute the longitude based on an
    # assumed latitude. here, the estimated longitude is to start the root-finding algorithm
    est_true_H = (Hs.deg + dip_correction(h).deg + index_correction().deg + atmo_correction(Hs).deg)
    #print('Finding Zo for true sextant angle of ' + '{:.3f}'.format(est_true_H) + ' degrees...')
    start_H = compute_Zo(latA, lonA, GHA, DEC).deg
    #print('Starting from value ' + '{:.3f}'.format(start_H) + ' degrees...')
    fz = lambda x: compute_Zo(latA, Angle(x,unit=u.deg), GHA, DEC).deg - (Hs.deg + dip_correction(h).deg + index_correction().deg + atmo_correction(Hs).deg)
    sp_out = scipy.optimize.brentq(fz, lonA.deg-5, lonA.deg+5,maxiter=100)

    return Angle(sp_out, unit=u.deg)

print(compute_Zo(Angle('17d04m'),Angle('-25d48m'),Angle('93d42.2m'),Angle('-0d25.1m')).to_string(unit=u.degree, sep=('deg', 'm', 's')))
print(compute_Zo(Angle('17d04m'),Angle('-25d49m'),Angle('356.34d'),Angle('-16d44.4m')).to_string(unit=u.degree, sep=('deg', 'm', 's')))

print(dip_correction(10*ureg.feet))

print(atmo_correction(Angle('13d')))

print(est_longitude(Angle('17d4m'), Angle('-25d48m'), Angle('93d42.2m'), Angle('-0d25.1m'), Angle('21d9m5s'), 10*ureg.feet))
print(est_longitude(Angle('17d4m'), Angle('-25d49m'), Angle('356.34d'), Angle('-16d44.4m'), Angle('45d32.5m'), 10*ureg.feet))

20deg56m37.8696s
45deg25m55.8043s
-0d03m04.1525s
-0d04m11.7869s
-25d52m46.7836s
-25d48m39.1369s


In [86]:
#plot run of sun sights

import matplotlib
import matplotlib.pyplot as plt

sun_sights = [
    [10*3600+11*60+17.0, Angle('51d07.3m')],
    [10*3600+12*60+13, Angle('51d22.1m')],
    [10*3600.0+13*60+8, Angle('51d29.8m')],
    [10*3600.0+13*60+55.0, Angle('51d39.5m')],
    [10*3600+15*60+5.0, Angle('51d48.1m')]
]
print(len(sun_sights))
# process list into 2d array

sun_sights_processed = np.zeros((len(sun_sights),2))
for i in range(len(sun_sights)):
    sun_sights_processed[i,0] = sun_sights[i][0]
    sun_sights_processed[i,1] = sun_sights[i][1].deg
    
plt.plot(sun_sights_processed[:,0],sun_sights_processed[:,1])
plt.show()

5
